In [1]:
import torch
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

sys.path.append('/home/noam.koren/multiTS/NFT/')
from dicts import data_to_num_vars_dict

sys.path.append('/home/noam.koren/multiTS/NFT/')
from training_functions import read_all_data_and_print_stats, train_model, evaluate_model, save_model

sys.path.append('/home/noam.koren/multiTS/NFT/models/baseline_models/')
from base_models import FC, MV_LSTM, CNN, TCN, TimeSeriesTransformer

data = 'air_quality'

NUM_OF_VARS = data_to_num_vars_dict[data]
LOOKBACK = 5
HORIZON = 1

BATCH_SIZE = 32
NUM_EPOCHS = 5
PLOT_EPOCH = NUM_EPOCHS

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
save_model_path = f"/home/noam.koren/multiTS/NFT/models/baseline_models/{data}/"

n_people = 600
n_rows = 700
    
class dataset(Dataset):
    def __init__(self, data, target):
        self.input = data
        self.target = target

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx): 
        return self.input[idx], self.target[idx]


In [ ]:
if data == 'ecg':
    data_path = f"/home/noam.koren/multiTS/NFT/data/{data}/{data}_{LOOKBACK}l_{HORIZON}h_{n_people}people_{n_rows}rows/"
else:
    data_path = f"/home/noam.koren/multiTS/NFT/data/{data}/{data}_{LOOKBACK}l_{HORIZON}h/"

train_X, train_y, val_X, val_y, test_X, test_y = read_all_data_and_print_stats(data_path=data_path)


In [ ]:
fc_model = FC(input_dim=NUM_OF_VARS, 
              output_dim=NUM_OF_VARS,
              horizon=HORIZON, 
              device=device).to(device)

train_model(dataset=dataset, 
            epochs=NUM_EPOCHS, 
            model=fc_model, 
            train_X=train_X.to(device), train_y=train_y.to(device),
            val_X=val_X.to(device), val_y=val_y.to(device), 
            device=device, 
            lookback=LOOKBACK,
            horizon=HORIZON, 
            n_vars=NUM_OF_VARS, 
            batch_size=BATCH_SIZE, 
            print_epoch=PLOT_EPOCH, 
            path_to_save_prediction_plots=None)

evaluate_model(fc_model, train_X, train_y, val_X, val_y, test_X, test_y)

save_model(fc_model, "fc", data, save_model_path, LOOKBACK, HORIZON, n_people, n_rows)

In [ ]:
lstm_model = MV_LSTM(num_vars=NUM_OF_VARS, 
                     lookback=LOOKBACK, 
                     horizon=HORIZON, 
                     hidden_dim=50, 
                     num_layers=2, 
                     device=device).to(device)

train_model(dataset=dataset, 
            epochs=NUM_EPOCHS, 
            model=lstm_model, 
            train_X=train_X.to(device), train_y=train_y.to(device),
            val_X=val_X.to(device), val_y=val_y.to(device), 
            device=device, 
            lookback=LOOKBACK,
            horizon=HORIZON, 
            n_vars=NUM_OF_VARS, 
            batch_size=BATCH_SIZE, 
            print_epoch=PLOT_EPOCH, 
            path_to_save_prediction_plots=None)

evaluate_model(lstm_model, train_X, train_y, val_X, val_y, test_X, test_y)

save_model(lstm_model, "lstm", data, save_model_path, LOOKBACK, HORIZON, n_people, n_rows)

In [ ]:
cnn_model = CNN(lookback=LOOKBACK, 
                horizon=HORIZON, 
                num_vars=NUM_OF_VARS,
                device=device,
                num_channels=[16], 
                kernel_size=2, 
                dropout=0.2).to(device)

train_model(dataset=dataset,
            epochs=NUM_EPOCHS,
            model=cnn_model, 
            train_X=train_X.to(device), train_y=train_y.to(device),
            val_X=val_X.to(device), val_y=val_y.to(device), 
            device=device, 
            lookback=LOOKBACK,
            horizon=HORIZON, 
            n_vars=NUM_OF_VARS, 
            batch_size=BATCH_SIZE, 
            print_epoch=PLOT_EPOCH, 
            path_to_save_prediction_plots=None
            )

evaluate_model(cnn_model, train_X, train_y, val_X, val_y, test_X, test_y)

save_model(cnn_model, "cnn", data, save_model_path, LOOKBACK, HORIZON, n_people, n_rows)

In [ ]:
tcn_model = TCN(lookback=LOOKBACK, 
                horizon=HORIZON, 
                num_vars=NUM_OF_VARS, 
                device=device,
                num_channels=[25, 50], 
                kernel_size=2, 
                dropout=0.2).to(device)

train_model(dataset=dataset,
            epochs=NUM_EPOCHS,
            model=tcn_model, 
            train_X=train_X.to(device), train_y=train_y.to(device),
            val_X=val_X.to(device), val_y=val_y.to(device), 
            device=device, 
            lookback=LOOKBACK,
            horizon=HORIZON, 
            n_vars=NUM_OF_VARS, 
            batch_size=BATCH_SIZE, 
            print_epoch=PLOT_EPOCH, 
            path_to_save_prediction_plots=None
            )

evaluate_model(tcn_model, train_X, train_y, val_X, val_y, test_X, test_y)

save_model(tcn_model, "tcn", data, save_model_path, LOOKBACK, HORIZON, n_people, n_rows)

In [ ]:
transformer_model = TimeSeriesTransformer(
    lookback=LOOKBACK,
    horizon=HORIZON,
    num_vars=NUM_OF_VARS,
    device=device,
    num_layers=1,
    num_heads=1,
    dim_feedforward=16,
).to(device)

train_model(dataset=dataset, 
            epochs=NUM_EPOCHS, 
            model=transformer_model, 
            train_X=train_X.to(device), train_y=train_y.to(device),
            val_X=val_X.to(device), val_y=val_y.to(device), 
            device=device, 
            lookback=LOOKBACK,
            horizon=HORIZON, 
            n_vars=NUM_OF_VARS, 
            batch_size=BATCH_SIZE, 
            print_epoch=PLOT_EPOCH, 
            path_to_save_prediction_plots=None)

evaluate_model(transformer_model, train_X, train_y, val_X, val_y, test_X, test_y)

save_model(transformer_model, "transformer", data, save_model_path, LOOKBACK, HORIZON, n_people, n_rows)